# Prepare FOI data for analysis

Prepare the FOI data for analysis, clean it, make convenience spreadsheets for eyeballing and cut it as needed for Flourish charts.

This assumes that you've pulled out the raw FOI Excel files to a CSV file per year in `./data/raw/`, using the naming convention `count-by-country-and-la-YYYY-MM.csv`.

In [15]:
import pandas as pd

## Concatenate and tidy the raw files

In [16]:
files = ["2010-01", "2012-01", "2014-01", "2016-01", "2018-01", "2020-01", "2021-08"]
df = pd.concat((pd.read_csv("./data/raw/count-by-country-and-la-%s.csv" % f)
                 .set_index(['COUNTRY_NAME','DIST_NAME_VAR', 'TENURE'])
        .add_suffix("_%s" % f) for f in files), axis=1)
df = df.reset_index()

Strip rogue whitespace.

In [17]:
df.COUNTRY_NAME = df.COUNTRY_NAME.str.strip()
df.DIST_NAME_VAR = df.DIST_NAME_VAR.str.strip()
df.TENURE = df.TENURE.str.strip()

### Tidy up countries

Tidy up Ireland, which looks like the only significant duplicate country name.

In [18]:
df.replace({"EIRE": "IRELAND"}, inplace=True)

### Tidy up local authorities

Tidy up local authority names - this happens because there were administrative changes between 2010 and 2021, mostly councils being consolidated into larger groups.

In [19]:
df.replace({
    "SOUTH BUCKS": "BUCKINGHAMSHIRE",
    "CHILTERN": "BUCKINGHAMSHIRE",
    "WYCOMBE": "BUCKINGHAMSHIRE",
    "AYLESBURY VALE": "BUCKINGHAMSHIRE",
    "NORTH DORSET": "DORSET",
    "WEST DORSET": "DORSET",
    "EAST DORSET": "DORSET",
    "WEYMOUTH AND PORTLAND": "DORSET",
    "PURBECK": "DORSET",
    "BOURNEMOUTH": "BOURNEMOUTH, CHRISTCHURCH AND POOLE",
    "CHRISTCHURCH": "BOURNEMOUTH, CHRISTCHURCH AND POOLE",
    "POOLE": "BOURNEMOUTH, CHRISTCHURCH AND POOLE",
    "CORBY": "NORTH NORTHAMPTONSHIRE",
    "EAST NORTHAMPTONSHIRE": "NORTH NORTHAMPTONSHIRE",
    "KETTERING": "NORTH NORTHAMPTONSHIRE",
    "WELLINGBOROUGH": "NORTH NORTHAMPTONSHIRE",
    "DAVENTRY": "WEST NORTHAMPTONSHIRE",
    "NORTHAMPTON": "WEST NORTHAMPTONSHIRE",
    "SOUTH NORTHAMPTONSHIRE": "WEST NORTHAMPTONSHIRE",
    "ST EDMUNDSBURY": "WEST SUFFOLK",
    "FOREST HEATH": "WEST SUFFOLK",
    "SUFFOLK COASTAL": "EAST SUFFOLK",
    "WAVENEY": "EAST SUFFOLK",
    "SHEPWAY": "FOLKESTONE AND HYTHE",
    "TAUNTON DEANE": "SOMERSET WEST AND TAUNTON",
    "WEST SOMERSET": "SOMERSET WEST AND TAUNTON",
    "PENWITH": "CORNWALL",
    "KERRIER": "CORNWALL",
    "CARRICK": "CORNWALL",
    "RESTORMEL": "CORNWALL",
    "CARADON": "CORNWALL",
    "NORTH CORNWALL": "CORNWALL",
    "DURHAM": "COUNTY DURHAM",
    "EASINGTON": "COUNTY DURHAM",
    "SEDGEFIELD": "COUNTY DURHAM",
    "TEESDALE": "COUNTY DURHAM",
    "WEAR VALLEY": "COUNTY DURHAM",
    "DERWENTSIDE": "COUNTY DURHAM",
    "CHESTER-LE-STREET": "COUNTY DURHAM",
    "BLYTH VALLEY": "NORTHUMBERLAND",
    "WANSBECK": "NORTHUMBERLAND",
    "CASTLE MORPETH": "NORTHUMBERLAND",
    "TYNEDALE": "NORTHUMBERLAND",
    "ALNWICK": "NORTHUMBERLAND",
    "BERWICK-UPON-TWEED": "NORTHUMBERLAND",
    "NORTH SHROPSHIRE": "SHROPSHIRE",
    "OSWESTRY": "SHROPSHIRE",
    "SHREWSBURY AND ATCHAM": "SHROPSHIRE",
    "SOUTH SHROPSHIRE": "SHROPSHIRE",
    "BRIDGNORTH": "SHROPSHIRE",
    "SALISBURY": "WILTSHIRE",
    "WEST WILTSHIRE": "WILTSHIRE",
    "KENNET": "WILTSHIRE",
    "NORTH WILTSHIRE": "WILTSHIRE",
    "ELLESMERE PORT AND NESTON": "CHESHIRE WEST AND CHESTER",
    "VALE ROYAL": "CHESHIRE WEST AND CHESTER",
    "CHESTER": "CHESHIRE WEST AND CHESTER",
    "CREWE AND NANTWICH": "CHESHIRE EAST",
    "CONGLETON": "CHESHIRE EAST",
    "MACCLESFIELD": "CHESHIRE EAST",
    "SOUTH BEDFORDSHIRE": "CENTRAL BEDFORDSHIRE",
    "MID BEDFORDSHIRE": "CENTRAL BEDFORDSHIRE",
    "BEDFORDSHIRE": "CENTRAL BEDFORDSHIRE"
}, inplace=True)

In [20]:
df = df.groupby(["COUNTRY_NAME", "DIST_NAME_VAR", "TENURE"]).sum().reset_index()

In [21]:
df.fillna(0, inplace=True)

In [22]:
for f in files:
    df["TITLES_%s" % f.replace("-", "_")] = df["NO_TITLES_%s" % f].astype(int)
    df.drop("NO_TITLES_%s" % f, axis=1, inplace=True)
df.rename(columns={"COUNTRY_NAME": "COUNTRY", "DIST_NAME_VAR": "DIST"}, inplace=True)

In [23]:
df.head()

,COUNTRY,DIST,TENURE,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08
0,AFGHANISTAN,ASHFIELD,L,0,0,0,0,0,1,1
1,AFGHANISTAN,EAST RIDING OF YORKSHIRE,F,1,1,1,1,1,1,1
2,ALBANIA,CORNWALL,L,0,0,0,0,0,1,1
3,ALBANIA,ENFIELD,L,0,0,0,0,0,1,0
4,ALBANIA,LAMBETH,L,0,0,0,0,0,0,1


In [24]:
df.to_csv("./data/all_data.csv", index=False)

## Create a country-only version

It's useful to have a separate CSV file with the counts grouped by countries.

In [26]:
df_by_country = df.groupby("COUNTRY").sum().reset_index()
df_by_country["abs_change_10_21"] = df_by_country["TITLES_2021_08"] - df_by_country["TITLES_2010_01"]
df_by_country["percent_change_10_21"] = df_by_country["abs_change_10_21"] / df_by_country["TITLES_2010_01"] * 100.0

In [27]:
df_by_country.head()

,COUNTRY,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08,abs_change_10_21,percent_change_10_21
0,AFGHANISTAN,1,1,1,1,1,2,2,1,100.000000
1,ALBANIA,0,0,0,0,0,4,5,5,inf
2,ALDERNEY,31,37,51,49,52,53,47,16,51.612903
3,ALGERIA,16,25,37,37,40,51,55,39,243.750000
4,ANDORRA,25,46,75,89,95,96,100,75,300.000000


In [28]:
df_by_country.sort_values("TITLES_2021_08", ascending=False).to_csv("./data/by_country.csv", index=False)

Make a pivoted version, for use in the Flourish bar chart.

In [29]:
df_by_country\
    .drop(["abs_change_10_21", "percent_change_10_21"], axis=1)\
    .sort_values("TITLES_2021_08", ascending=False).set_index('COUNTRY')\
    .transpose().to_csv("./data/barchart.csv", encoding='utf-8')

## Create a local authority-only version

In [30]:
df_by_dist = df.groupby("DIST").sum().reset_index()
df_by_dist["abs_change_10_21"] = df_by_dist["TITLES_2021_08"] - df_by_dist["TITLES_2010_01"]
df_by_dist["percent_change_10_21"] = df_by_dist["abs_change_10_21"] / df_by_dist["TITLES_2010_01"] * 100.0
df_by_dist.sort_values("TITLES_2021_08", ascending=False).to_csv("./data/by_dist.csv", index=False)

## Create a country and local authority version

For analysis and for the Flourish table.

In [45]:
df_by_country_and_dist = df.groupby(["COUNTRY", "DIST"]).sum().reset_index()
df_by_country_and_dist["abs_change_10_21"] = \
    df_by_country_and_dist["TITLES_2021_08"] - df_by_country_and_dist["TITLES_2010_01"]
df_by_country_and_dist["percent_change_10_21"] = \
    df_by_country_and_dist["abs_change_10_21"] / df_by_country_and_dist["TITLES_2010_01"] * 100.0
df_by_country_and_dist\
    .sort_values("TITLES_2021_08", ascending=False)\
    .to_csv("./data/by_country_and_dist.csv", index=False)

## Have a quick look at tenures

We don't analyse freeholds/leaseholds in the main analysis, have a quick look here. Freeholds used to outnumber leaseholds, now the split is roughly 50/50. This probably indicates more flats being bought in recent years.

In [31]:
df_by_tenure = df.groupby("TENURE").sum().reset_index()
df_by_tenure.sort_values("TITLES_2021_08", ascending=False).head()

,TENURE,TITLES_2010_01,TITLES_2012_01,TITLES_2014_01,TITLES_2016_01,TITLES_2018_01,TITLES_2020_01,TITLES_2021_08
0,F,50747,67976,84926,100991,112356,121725,123621
1,L,37063,52066,67377,83009,103213,117929,123395
